# Экспериментальная часть

In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import string

import scipy.io as sio
from scipy.special import rel_entr, kl_div
from scipy.stats import entropy, ks_2samp
from scipy.spatial.distance import jensenshannon
from skmultiflow.drift_detection import DDM, PageHinkley, ADWIN
from skmultiflow.data import ConceptDriftStream
import datetime
from sklearn.datasets import load_svmlight_file
import matplotlib.pyplot as plt
import os

In [99]:
count_vect = CountVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
tfidf_vect = TfidfVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
porter_stemmer = nltk.stem.PorterStemmer()
tok = count_vect.build_tokenizer()
clf = LogisticRegression(solver='liblinear', C = 2.8, penalty = 'l1')
samples_number = 4

In [3]:
df_all = pd.read_csv('../Amazon_Electronics/Electronics_5.csv')
df_all

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1,"Major issues, only excuses for support",1317254400,"09 29, 2011"
...,...,...,...,...,...,...,...,...,...
1689183,A34BZM6S9L7QI4,B00LGQ6HL8,"Candy Cane ""Is it just me?""","[1, 1]",Burned these in before listening to them for a...,5,Boom -- Pop -- Pow. These deliver.,1405555200,"07 17, 2014"
1689184,A1G650TTTHEAL5,B00LGQ6HL8,"Charles Spanky ""Zumina Reviews""","[0, 0]",Some people like DJ style headphones or earbud...,5,"Thin and light, without compromising on sound ...",1405382400,"07 15, 2014"
1689185,A25C2M3QF9G7OQ,B00LGQ6HL8,Comdet,"[0, 0]",I&#8217;m a big fan of the Brainwavz S1 (actua...,5,Same form factor and durability as the S1 with...,1405555200,"07 17, 2014"
1689186,A1E1LEVQ9VQNK,B00LGQ6HL8,J. Chambers,"[0, 0]","I've used theBrainwavz S1 In Ear Headphones, a...",5,Superb audio quality in a very comfortable set...,1405641600,"07 18, 2014"


In [4]:
df_all.reviewTime = pd.to_datetime(df_all.reviewTime, format="%m %d, %Y")
df_all = df_all.sort_values(by = 'reviewTime').reset_index(drop = True)
df_all["bin_label"] = df_all.overall > 3
df_all["words"] = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), df_all.reviewText.apply(lambda x: np.str_(x)))))
df_all

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,bin_label,words
0,A1ABVP0DV1ZN89,B00000JBAT,Adam Sacks,"[0, 0]",The RIO rocks! It is so great that Diamond Mul...,5,WOW! THE BEST THING TO HAPPEN TO MUSIC!,929232000,1999-06-13,True,the rio rock it is so great that diamond multi...
1,A6ILK3FXYH595,B00000J4FS,kc2kth,"[1, 1]",I have found a few places that will do memory ...,5,After-market Memory Upgrades,931132800,1999-07-05,True,have found few place that will do memori upgra...
2,A7RV1KU5O0II9,B00000JFMK,"David S. Saunders ""Equal parts tech geek, bra...","[0, 0]",I love this player! I travel frequently and th...,5,Incredible screen!,931392000,1999-07-08,True,love thi player travel frequent and the l50 ke...
3,AJ6TMOHHFJJAJ,B00000JMO4,Ali Navaie,"[5, 5]",If you are looking for the best camcorder in t...,5,Simply the best in the market --- Get it while...,932688000,1999-07-23,True,if you are look for the best camcord in the ma...
4,A1JTSRG8SU4VFO,B00000JFE3,L. Goff,"[0, 0]",The &quot;lip-synch&quot; problem has shown up...,5,Own the Sony 550D model,932688000,1999-07-23,True,the quot lip synch quot problem ha shown up on...
...,...,...,...,...,...,...,...,...,...,...,...
1689183,A9DH6MLZBGUMO,B00J8ZJCUW,Jules,"[0, 0]",I had to order two of these - The GLAZZ was so...,5,10 stars! The PERFECT protection for your iPad...,1406073600,2014-07-23,True,had to order two of these the glazz wa so well...
1689184,A1SLHNA3FV9U9E,B00HZVPD72,ashtangakasha,"[0, 0]","Tiny, simple, excellent sound quality, easy to...",4,"Tiny, simple, and effective.",1406073600,2014-07-23,True,tini simpl excel sound qualiti easi to set up ...
1689185,A2YQ9AX4GOTA0S,B00BOEG08M,NaN,"[0, 0]",Works great. Very convenient.,5,Five Stars,1406073600,2014-07-23,True,work great veri conveni
1689186,A2MUTLXDSV3JRC,B00IERCFFO,"MBW66 ""Pilcopata Man""","[0, 0]",Elegant trim case for iPad Air. No zipper case...,5,Slim case for iPad Air,1406073600,2014-07-23,True,eleg trim case for ipad air no zipper case for...


In [5]:
df_by_years = {}
for year in df_all.reviewTime.dt.year.unique().tolist():
    df_by_years[year] = df_all[df_all.reviewTime.dt.year == year].reset_index(drop = True)
    print(year, len(df_by_years[year]))
    print(df_by_years[year].bin_label.value_counts(), '\n')

1999 72
True     61
False    11
Name: bin_label, dtype: int64 

2000 817
True     686
False    131
Name: bin_label, dtype: int64 

2001 1609
True     1292
False     317
Name: bin_label, dtype: int64 

2002 2315
True     1802
False     513
Name: bin_label, dtype: int64 

2003 3547
True     2663
False     884
Name: bin_label, dtype: int64 

2004 5159
True     3655
False    1504
Name: bin_label, dtype: int64 

2005 9638
True     6994
False    2644
Name: bin_label, dtype: int64 

2006 15447
True     11528
False     3919
Name: bin_label, dtype: int64 

2007 35976
True     28404
False     7572
Name: bin_label, dtype: int64 

2008 49872
True     39460
False    10412
Name: bin_label, dtype: int64 

2009 70666
True     55304
False    15362
Name: bin_label, dtype: int64 

2010 103797
True     80475
False    23322
Name: bin_label, dtype: int64 

2011 173395
True     134860
False     38535
Name: bin_label, dtype: int64 

2012 282942
True     224559
False     58383
Name: bin_label, dtype: int64 

2

In [105]:
data_by_year_splitted = {}
for year in range(2007, 2015):
    data_tmp = df_by_years[year]
    data_list, labels_list = [], []
    
    for i in range(samples_number):
        data, data_tmp = train_test_split(data_tmp, train_size = 8000, stratify = data_tmp.bin_label)
        data_list.append(data.words)
        labels_list.append(data.bin_label)
        
    data_by_year_splitted[year] = {'data': data_list, 'labels': labels_list}
    print(year)

2007
2008
2009
2010
2011
2012
2013
2014


In [106]:
df_accuracy = pd.DataFrame(np.zeros([2015-2007, 2015-2007]),index=range(2007, 2015), columns=range(2007, 2015))
df_accuracy

,2007,2008,2009,2010,2011,2012,2013,2014
2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
for year_train in range(2007, 2015):
    for test_index in range(samples_number):
        tfidf_vect = TfidfVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
        clf = LogisticRegression(solver='liblinear', C = 2.8, penalty = 'l1')
        for train_index in range(samples_number):
            if test_index != train_index:
                clf.fit(tfidf_vect.fit_transform(data_by_year_splitted[year_train]['data'][train_index]), data_by_year_splitted[year_train]['labels'][train_index])
        for year_test in range(year_train, 2015):
            print("Train in {0}, test in {1}, test sample {2}".format(year_train, year_test, test_index))
            df_accuracy.at[year_train, year_test] += accuracy_score(clf.predict(tfidf_vect.transform(data_by_year_splitted[year_test]['data'][test_index])),\
                                                                          data_by_year_splitted[year_test]['labels'][test_index])
df_accuracy = df_accuracy/samples_number
df_accuracy[df_accuracy.eq(0)] = np.nan

Train in 2007, test in 2007, test sample 0
Train in 2007, test in 2008, test sample 0
Train in 2007, test in 2009, test sample 0
Train in 2007, test in 2010, test sample 0
Train in 2007, test in 2011, test sample 0
Train in 2007, test in 2012, test sample 0
Train in 2007, test in 2013, test sample 0
Train in 2007, test in 2014, test sample 0
Train in 2007, test in 2007, test sample 1
Train in 2007, test in 2008, test sample 1
Train in 2007, test in 2009, test sample 1
Train in 2007, test in 2010, test sample 1
Train in 2007, test in 2011, test sample 1
Train in 2007, test in 2012, test sample 1
Train in 2007, test in 2013, test sample 1
Train in 2007, test in 2014, test sample 1
Train in 2007, test in 2007, test sample 2
Train in 2007, test in 2008, test sample 2
Train in 2007, test in 2009, test sample 2
Train in 2007, test in 2010, test sample 2
Train in 2007, test in 2011, test sample 2
Train in 2007, test in 2012, test sample 2
Train in 2007, test in 2013, test sample 2
Train in 20

In [109]:
ur_style = (df_accuracy
            .style
            .background_gradient(cmap='Greens', axis=0)
            .highlight_null('white')
            .format("{:.2%}", na_rep=""))
ur_style

,2007,2008,2009,2010,2011,2012,2013,2014
2007,85.84%,86.03%,85.69%,85.15%,84.98%,85.96%,86.99%,87.22%
2008,,86.19%,85.68%,85.47%,85.28%,86.19%,87.43%,87.18%
2009,,,85.89%,85.63%,85.48%,86.00%,87.27%,87.32%
2010,,,,85.54%,85.33%,86.25%,87.41%,87.48%
2011,,,,,85.72%,86.43%,87.38%,87.50%
2012,,,,,,86.33%,87.52%,87.57%
2013,,,,,,,86.92%,87.30%
2014,,,,,,,,87.53%
